In [1]:
import pandas as pd
train_data_path = "data/train_AUpWtIz/train.csv"
campaign_data_path = "data/train_AUpWtIz/campaign_data.csv"
coupon_item_mapping_data_path = "data/train_AUpWtIz/coupon_item_mapping.csv"
customer_demographics_data_path = "data/train_AUpWtIz/customer_demographics.csv"
customer_transaction_data_path = "data/train_AUpWtIz/customer_transaction_data.csv"
item_data_path = "data/train_AUpWtIz/item_data.csv"

train_df = pd.read_csv(train_data_path)
campaign_df = pd.read_csv(campaign_data_path)
coupon_item_df = pd.read_csv(coupon_item_mapping_data_path)
customer_demo_df = pd.read_csv(customer_demographics_data_path)
customer_trans_df = pd.read_csv(customer_transaction_data_path)
item_df =  pd.read_csv(item_data_path)

In [2]:
train_df.describe()

,id,campaign_id,coupon_id,customer_id,redemption_status
count,78369.000000,78369.000000,78369.000000,78369.000000,78369.000000
mean,64347.975449,13.974441,566.363243,787.451888,0.009302
std,37126.440855,8.019215,329.966054,456.811339,0.095999
min,1.000000,1.000000,1.000000,1.000000,0.000000
25%,32260.000000,8.000000,280.000000,399.000000,0.000000
50%,64318.000000,13.000000,597.000000,781.000000,0.000000
75%,96577.000000,13.000000,857.000000,1190.000000,0.000000
max,128595.000000,30.000000,1115.000000,1582.000000,1.000000


In [3]:
print(train_df.isnull().sum())
print("-------------------------------")
print(campaign_df.isnull().sum())
print("-------------------------------")
print(coupon_item_df.isnull().sum())
print("-------------------------------")
print(customer_demo_df.isnull().sum())
print("-------------------------------")
print(customer_trans_df.isnull().sum())
print("-------------------------------")
print(item_df.isnull().sum())
print("-------------------------------")

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
dtype: int64
-------------------------------
campaign_id       0
campaign_type     0
start_date        0
end_date          0
correction       23
dtype: int64
-------------------------------
coupon_id    0
item_id      0
dtype: int64
-------------------------------
customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64
-------------------------------
date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64
-------------------------------
item_id       0
brand         0
brand_type    0
category      0
dtype: int64
-------------------------------


In [4]:
customer_demo_df = customer_demo_df[['customer_id', 'age_range', 'rented', 'family_size','income_bracket']]

In [5]:
#train_df has all the numeric columns
import datetime
campaign_df_1 = campaign_df.copy()
campaign_df_1['start_date'] = campaign_df_1['start_date'].apply(lambda x: datetime.datetime.strptime(x,"%d/%m/%y"))
campaign_df_1['end_date'] = campaign_df_1['end_date'].apply(lambda x: datetime.datetime.strptime(x,"%d/%m/%y"))
campaign_df_1['month'] = campaign_df_1['start_date'].dt.month


In [6]:
#check the campaign dataframe
campaign_df.select_dtypes(exclude = ["number"])
#it gives three col "campaign_type" ,"start_date","end_date" are not numbers.

#USING GETDUMMIES FUNCTION
campaign_df_1 = pd.get_dummies(campaign_df_1,columns = ['campaign_type'])
#df_with_dummies = pd.get_dummies( df, columns = cols_to_transform )

#check the duration and month from the dates.
campaign_df_1['total_days'] = campaign_df_1['end_date'].values - campaign_df_1['start_date'].values


In [7]:
#Desired columns from campaign_df csv = ['month',campaign_type_X,campaign_type_Y,total_days]
campaign_data_df = campaign_df_1.loc[:,["campaign_id",'month','campaign_type_X','campaign_type_Y','total_days']]
campaign_data_df['total_days'] = campaign_data_df['total_days'].dt.days
campaign_data_df.to_csv("campaign_data_df.csv")

In [8]:
#lets numalarise   coupon_item_mapping_data_path csv 

#nothing is there to convert.

In [9]:
#lets numalarise customer_demographics_data_path csv
customer_demo_df_1 = customer_demo_df.copy()
list1 = ['customer_id', 'age_range', 'rented', 'family_size', 'income_bracket']
list2 = ['age_range','rented','income_bracket']
#rented is categorical 
#age range is categorical
#income_bracket is categorical data 
#convert them to categorical variable.
customer_demo_df_1 = pd.get_dummies(customer_demo_df_1, columns = list2)
customer_demo_df_1.to_csv('customer_demo_df_1.csv')

In [10]:
#lets move to customer_transaction_data_path
customer_transaction_data_path = "data/train_AUpWtIz/customer_transaction_data.csv"
customer_trans_df = pd.read_csv(customer_transaction_data_path)
#nothing to change here.

In [11]:
#the last one is item_data_path
item_data_path = "data/train_AUpWtIz/item_data.csv"
item_df =  pd.read_csv(item_data_path)
item_df_1 = item_df.loc[:,['item_id', 'brand_type', 'category']]
list3 = ['brand_type', 'category']
#convert above list to categorical variables.
item_df_1 = pd.get_dummies(item_df_1, columns = list3)
item_df_1.to_csv("item_df_1.csv")

In [12]:
# lets group them together customer_demo_df and customer_trans_df
#combine train and campaign type.
#two dataframes are -
import pandas as pd
demogra_df = pd.read_csv('customer_demo_df_1.csv')
c_trans_df = pd.read_csv('customer_transaction_data.csv')

# df1 = c_trans_df.join(demogra_df, on = 'customer_id')
#df1 = pd.merge(demogra_df,c_trans_df,on='customer_id',how = 'left')
b = c_trans_df.set_index("customer_id")
df1 = demogra_df.join(b,lsuffix='L_',rsuffix='R_',on="customer_id")
del df1['Unnamed: 0']
del df1['date']
df1.isnull().sum()
df1

,customer_id,family_size,age_range_18-25,age_range_26-35,age_range_36-45,age_range_46-55,age_range_56-70,age_range_70+,rented_0,rented_1,...,income_bracket_8,income_bracket_9,income_bracket_10,income_bracket_11,income_bracket_12,item_id,quantity,selling_price,other_discount,coupon_discount
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,4953,1,142.12,0.00,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,5626,1,106.50,0.00,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,7808,1,99.38,-42.74,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,8107,1,256.11,-46.31,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,8307,1,89.05,-17.45,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,14054,1,53.07,0.00,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,17047,2,106.50,-106.50,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,17559,1,49.51,-17.81,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,18807,1,56.64,0.00,0.00
0,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,0,20811,1,35.62,-17.45,0.00


In [11]:
#lets merge train data and campaign data
t_df = pd.read_csv('train.csv')
c_df = pd.read_csv('campaign_data_df.csv')
b = c_df.set_index("campaign_id")
df2 = t_df.join(c_df, on = 'campaign_id',lsuffix='L', rsuffix='R')
#df2 = pd.merge(c_df,t_df, on = 'campaign_id',how='right')
del df2['Unnamed: 0']
del df2['id']
#df2['customer_id'].nunique() = 1428
df2.dropna(inplace = True)
df2.isnull().sum()

campaign_idL         0
coupon_id            0
customer_id          0
redemption_status    0
campaign_idR         0
month                0
campaign_type_X      0
campaign_type_Y      0
total_days           0
dtype: int64

In [13]:
#lets combine df1 and df2 on custmor id
#df3 = pd.merge(df2,df1,how='left',on='customer_id')
b = df1.set_index("customer_id")
df3 = df2.join(b,lsuffix= 'L_',rsuffix = "R_",on="customer_id")
df3.to_csv('df3.csv')

MemoryError: Unable to allocate array with shape (22, 45226728) and data type float64